In [5]:
# =============================================================================
# 1. IMPORTACIONES
# =============================================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from scipy.stats import randint, uniform

# MLflow para seguimiento
import mlflow
import mlflow.sklearn

# Preprocesamiento y Pipeline
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder, RobustScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Modelos
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

# Métricas y Evaluación
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_auc_score,
    roc_curve, auc, precision_recall_curve
)
from sklearn.preprocessing import label_binarize
from sklearn.inspection import PartialDependenceDisplay

# Ignorar advertencias para una salida más limpia (opcional)
import warnings
warnings.filterwarnings('ignore')


# =============================================================================
# 2. CLASE DE GESTIÓN DE DATOS
# =============================================================================

class DataManager:
    """
    Encapsula la carga, limpieza y preparación de los datos.
    """
    def __init__(self, file_path, sep=None):
        """
        Inicializa el gestor de datos.

        :param file_path: Ruta al archivo CSV.
        :param sep: Separador del CSV (opcional).
        """
        self.file_path = file_path
        self.sep = sep
        print(f"DataManager inicializado con el archivo: {file_path}")

    def load_and_clean_data(self):
        """
        Carga el dataset, lo limpia y elimina duplicados y columnas nulas.
        """
        print("Iniciando carga y limpieza de datos...")
        # Cargar dataset original
        df_raw = pd.read_csv(self.file_path, sep=self.sep, engine="python", encoding="utf-8")
        df = df_raw.copy()

        # Estandarizar nulos y espacios
        df = df.replace(r"^\s*$", np.nan, regex=True)
        df = df.replace({"NA": np.nan, "N/A": np.nan, "na": np.nan, "NaN": np.nan})

        # Recortar strings
        obj_cols = df.select_dtypes(include=["object"]).columns.tolist()
        for c in obj_cols:
            df[c] = df[c].astype(str).str.strip()

        # Eliminar duplicados
        duplicates_before = df.duplicated().sum()
        df = df.drop_duplicates()
        print(f"Se eliminaron {duplicates_before} duplicados.")

        # Eliminar columnas 100% nulas
        all_null = [c for c in df.columns if df[c].isna().all()]
        if all_null:
            df = df.drop(columns=all_null)
            print(f"Columnas eliminadas (100% nulas): {all_null}")
        else:
            print("No se encontraron columnas 100% nulas.")


        print("Carga y limpieza de datos completada.")
        return df

    def split_data(self, df, target_col, test_size=0.2, random_state=42):
        """
        Divide los datos en características (X) y objetivo (y),
        y luego en conjuntos de entrenamiento y prueba.

        :param df: DataFrame limpio.
        :param target_col: Nombre de la columna objetivo.
        :param test_size: Proporción del dataset para el conjunto de prueba.
        :param random_state: Semilla para reproducibilidad.
        :return: X_train, X_test, y_train, y_test
        """
        print("Dividiendo datos en entrenamiento y prueba...")
        X = df.drop(target_col, axis=1)
        y = df[target_col]
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=test_size, random_state=random_state, stratify=y
        )
        return X_train, X_test, y_train, y_test

    def encode_target(self, y_train, y_test):
        """
        Codifica la variable objetivo (y) usando LabelEncoder.

        :param y_train: Objetivo de entrenamiento.
        :param y_test: Objetivo de prueba.
        :return: y_train_encoded, y_test_encoded, label_encoder (el objeto ajustado)
        """
        print("Codificando variable objetivo...")
        label_encoder = LabelEncoder()
        y_train_encoded = label_encoder.fit_transform(y_train)
        y_test_encoded = label_encoder.transform(y_test)
        return y_train_encoded, y_test_encoded, label_encoder


# =============================================================================
# 3. CLASE DE FÁBRICA DE PIPELINES
# =============================================================================

class ModelPipelineFactory:
    """
    Construye y entrena pipelines de modelo con búsqueda de hiperparámetros.
    """
    def __init__(self):
        self.preprocessor = None

    def _create_preprocessor(self, X_train):
        """
        Crea un preprocesador (ColumnTransformer) basado en los tipos de
        columnas de X_train.

        :param X_train: DataFrame de características de entrenamiento.
        :return: Objeto ColumnTransformer.
        """
        # Identificar tipos de columnas desde X_train
        numeric_features = X_train.select_dtypes(include=np.number).columns
        categorical_features = X_train.select_dtypes(
            include=['object', 'category']
        ).columns

        print(f"Preprocesador: {len(numeric_features)} features numéricas y {len(categorical_features)} categóricas.")

        # Pipeline para variables numéricas
        numeric_transformer = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='constant', fill_value=-20)),
            ('scaler', RobustScaler())
        ])

        # Pipeline para variables categóricas
        categorical_transformer = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='constant', fill_value='Missing')),
            ('onehot', OneHotEncoder(
                handle_unknown='ignore', sparse_output=False, drop='first'
            ))
        ])

        # Crear el ColumnTransformer
        self.preprocessor = ColumnTransformer(transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)
        ], remainder='passthrough')

        return self.preprocessor

    def create_and_tune_model(self, X_train, y_train, classifier, param_grid, search_options):
        """
        Crea un pipeline completo y lo entrena usando RandomizedSearchCV.

        :param X_train: Características de entrenamiento.
        :param y_train: Objetivo de entrenamiento (codificado).
        :param classifier: El objeto clasificador (ej. XGBClassifier()).
        :param param_grid: Diccionario de hiperparámetros para la búsqueda.
        :param search_options: Diccionario de opciones para RandomizedSearchCV.
        :return: Objeto RandomizedSearchCV ajustado.
        """

        # 1. Crear el preprocesador si aún no existe
        if self.preprocessor is None:
            self.preprocessor = self._create_preprocessor(X_train)

        # 2. Crear el pipeline principal
        model_pipeline = Pipeline(steps=[
            ('preprocessor', self.preprocessor),
            ('classifier', classifier)
        ])

        # 3. Configurar y ejecutar la búsqueda aleatoria
        random_search = RandomizedSearchCV(
            model_pipeline,
            param_distributions=param_grid,
            **search_options  # Desempaqueta n_iter, cv, scoring, etc.
        )

        model_name = classifier.__class__.__name__
        print(f"Iniciando RandomizedSearchCV para {model_name}...")
        random_search.fit(X_train, y_train)
        print(f"Búsqueda completada para {model_name}.")

        return random_search


# =============================================================================
# 4. CLASE DE EVALUACIÓN DE MODELOS (MODIFICADA PARA MLFLOW)
# =============================================================================

class ModelEvaluator:
    """
    Encapsula la evaluación e interpretación de un modelo entrenado.
    Modificado para loguear artefactos en MLflow.
    """
    def __init__(self, best_model, X_test, y_test_original, y_test_encoded, label_encoder):
        """
        Inicializa el evaluador con el modelo y los datos de prueba.
        """
        self.model = best_model
        self.X_test = X_test
        self.y_test_original = y_test_original # 'y_test' (strings)
        self.y_test_encoded = y_test_encoded # 'y_test_encoded' (números)
        self.le = label_encoder
        self.class_labels = self.le.classes_
        self.model_name = self.model.named_steps['classifier'].__class__.__name__
        print(f"Evaluador listo para el modelo: {self.model_name}")

    def evaluate_classification(self):
        """
        Genera y muestra métricas de evaluación, matriz de confusión y curvas ROC/PR.
        Loguea los resultados como texto y figuras en MLflow.
        """
        # Realizar predicciones
        y_pred_encoded = self.model.predict(self.X_test)
        y_pred_proba = self.model.predict_proba(self.X_test)
        y_pred_original = self.le.inverse_transform(y_pred_encoded)

        # --- Reporte de Clasificación ---
        print("\n--- Reporte de Clasificación ---")
        reporte_str = classification_report(
            self.y_test_original, y_pred_original, target_names=self.class_labels
        )
        print(reporte_str)
        # Loguear reporte como texto en MLflow
        mlflow.log_text(reporte_str, "classification_report.txt")

        # --- Matriz de Confusión ---
        print("\n--- Matriz de Confusión ---")
        cm = confusion_matrix(self.y_test_original, y_pred_original, labels=self.class_labels)
        fig_cm = plt.figure(figsize=(10, 8))
        sns.heatmap(
            cm, annot=True, fmt='g', cmap='Blues',
            xticklabels=self.class_labels, yticklabels=self.class_labels
        )
        plt.xlabel('Predicción'); plt.ylabel('Valor Real')
        plt.title(f'Matriz de Confusión - {self.model_name}')
        # Loguear figura en MLflow
        mlflow.log_figure(fig_cm, "matriz_confusion.png")
        plt.close(fig_cm) # Cerrar la figura para no mostrarla en el notebook ahora

        # --- Curvas ROC y PR ---
        y_test_binarized = label_binarize(self.y_test_encoded, classes=range(len(self.class_labels)))
        n_classes = y_test_binarized.shape[1]

        fig_roc_pr, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))
        fig_roc_pr.suptitle(f"Curvas de Evaluación para {self.model_name}", fontsize=16)

        # Curva ROC AUC
        try:
            roc_auc = roc_auc_score(y_test_binarized, y_pred_proba, multi_class='ovr')
            print(f"\nROC AUC Score (One-vs-Rest): {roc_auc:.4f}")
            # MLflow autolog() usualmente captura 'roc_auc_score' si se llama
            # mlflow.log_metric("test_roc_auc_ovr", roc_auc)
            for i in range(n_classes):
                fpr, tpr, _ = roc_curve(y_test_binarized[:, i], y_pred_proba[:, i])
                ax1.plot(fpr, tpr, lw=2, label=f'{self.class_labels[i]} (AUC={auc(fpr, tpr):.2f})')
        except ValueError as e:
            print(f"Error al calcular ROC AUC (puede pasar en problemas binarios/formato): {e}")
            ax1.set_title("No se pudo generar Curva ROC")

        ax1.plot([0, 1], [0, 1], 'k--', lw=2)
        ax1.set_xlabel('Tasa de Falsos Positivos (FPR)'); ax1.set_ylabel('Tasa de Verdaderos Positivos (TPR)')
        ax1.set_title('Curva ROC Multiclase (One-vs-Rest)'); ax1.legend(loc="lower right"); ax1.grid(True)

        # Curva Precisión-Recall
        for i in range(n_classes):
            precision, recall, _ = precision_recall_curve(
                y_test_binarized[:, i], y_pred_proba[:, i]  # <--- ¡CORREGIDO!
            )
            ax2.plot(recall, precision, lw=2, label=f'Clase {self.class_labels[i]}')
        ax2.set_xlabel("Recall (Sensibilidad)"); ax2.set_ylabel("Precision")
        ax2.set_title("Curva Precisión-Recall Multiclase"); ax2.legend(loc="best"); ax2.grid(True)
        # Loguear figura en MLflow
        mlflow.log_figure(fig_roc_pr, "curvas_roc_pr.png")
        plt.close(fig_roc_pr)

    def plot_feature_importance(self, X_train):
        """
        Calcula y grafica la importancia de las variables del modelo.
        Loguea la figura en MLflow.
        """
        if not hasattr(self.model.named_steps['classifier'], 'feature_importances_'):
            print(f"\nEl modelo seleccionado ({self.model_name}) no tiene 'feature_importances_'.")
            return

        print("\n--- Importancia de Variables ---")
        model = self.model.named_steps['classifier']
        preprocessor = self.model.named_steps['preprocessor']

        try:
            # Obtener nombres de features del preprocesador
            numeric_features = X_train.select_dtypes(include=np.number).columns
            cat_features = X_train.select_dtypes(include=['object', 'category']).columns
            ohe_features = preprocessor.named_transformers_['cat'].named_steps['onehot'].get_feature_names_out(cat_features)
            all_features = np.concatenate([numeric_features, ohe_features])

            importances_df = pd.DataFrame({
                'feature': all_features,
                'importance': model.feature_importances_
            }).sort_values(by='importance', ascending=False)

            fig_fi = plt.figure(figsize=(12, 10))
            sns.barplot(x='importance', y='feature', data=importances_df.head(20), palette='viridis')
            plt.title(f'Top 20 Variables Importantes ({self.model_name})', fontsize=16)
            plt.xlabel('Importancia'); plt.ylabel('Variable'); plt.grid(axis='x', alpha=0.5)
            # Loguear figura en MLflow
            mlflow.log_figure(fig_fi, "importancia_variables.png")
            plt.close(fig_fi)

        except Exception as e:
            print(f"Error al calcular feature importance: {e}")

    def plot_partial_dependence(self, X_train, features_to_plot):
        """
        Genera y muestra las curvas de dependencia parcial (PDP) para las
        variables especificadas. Loguea las figuras en MLflow.

        :param X_train: DataFrame de entrenamiento.
        :param features_to_plot: Lista de nombres de columnas a graficar.
        """
        print(f"\n--- Generando Curvas de Dependencia Parcial para: {features_to_plot} ---")

        for feature in features_to_plot:
            n_classes = len(self.class_labels)
            n_cols = 3
            n_rows = math.ceil(n_classes / n_cols)
            fig_pdp, axes = plt.subplots(n_rows, n_cols, figsize=(15, 4 * n_rows), sharey=True)
            fig_pdp.suptitle(f'Dependencia Parcial de "{feature}" ({self.model_name})', fontsize=16, y=1.02)

            for i, class_name in enumerate(self.class_labels):
                ax = axes.flatten()[i]
                try:
                    PartialDependenceDisplay.from_estimator(
                        self.model,
                        X_train,
                        features=[feature],
                        target=i,
                        ax=ax
                    )
                    ax.set_title(f'Target: {class_name}')
                    ax.set_xlabel(feature)
                    ax.set_ylabel("Dependencia Parcial")
                except Exception as e:
                    print(f"Error al generar PDP para {feature}, clase {class_name}: {e}")
                    ax.set_title(f'Error al generar PDP para {class_name}')
                    ax.axis('off')

            # Ocultar ejes no utilizados
            for j in range(n_classes, len(axes.flatten())):
                axes.flatten()[j].axis('off')

            plt.tight_layout()
            plt.subplots_adjust(top=0.92)
            # Loguear figura en MLflow
            mlflow.log_figure(fig_pdp, f"pdp_{feature}.png")
            plt.close(fig_pdp)


# =============================================================================
# 5. BLOQUE PRINCIPAL DE EJECUCIÓN (MODIFICADO PARA MLFLOW)
# =============================================================================

def define_models_to_train():
    """
    Define la configuración de los modelos y sus hiperparámetros.
    Separar esto como una función mantiene limpio el bloque principal.
    """

    models_config = [
        {
            "name": "KNN",
            "classifier": KNeighborsClassifier(),
            "param_grid": {
                'classifier__n_neighbors': randint(3, 31),
                'classifier__weights': ['uniform', 'distance'],
                'classifier__metric': ['euclidean', 'manhattan', 'minkowski'],
            }
        },
        {
            "name": "Random Forest",
            "classifier": RandomForestClassifier(random_state=42),
            "param_grid": {
                'classifier__n_estimators': randint(100, 500),
                'classifier__max_depth': randint(5, 30),
                'classifier__min_samples_split': randint(2, 20),
                'classifier__min_samples_leaf': randint(1, 20),
                'classifier__max_features': ['sqrt', 'log2'],
            }
        },
        {
            "name": "XGBoost",
            "classifier": XGBClassifier(
                objective='multi:softmax', eval_metric='mlogloss',
                use_label_encoder=False, random_state=42
            ),
            "param_grid": {
                'classifier__n_estimators': randint(100, 500),
                'classifier__max_depth': randint(3, 10),
                'classifier__learning_rate': uniform(0.01, 0.3),
                'classifier__subsample': uniform(0.6, 0.4),
                'classifier__colsample_bytree': uniform(0.6, 0.4),
            }
        }
    ]
    return models_config



In [6]:
if __name__ == "__main__":

    # --- 1. Configuración Inicial ---
    SRC_PATH = '/content/drive/MyDrive/Pregrado - Posgrado - Trabajo/Maestría - Inteligencia Artificial Aplicada/11. MLOps/1. Primera etapa de proyecto/Modelado/obesity_estimation_original.csv'
    TARGET = 'NObeyesdad'

    print("Iniciando Proceso de ML Completo...")

    # --- 2. Preparación de Datos ---
    # Instanciar y usar el DataManager
    data_manager = DataManager(file_path=SRC_PATH)
    df_cleaned = data_manager.load_and_clean_data()

    X_train, X_test, y_train, y_test = data_manager.split_data(
        df_cleaned, TARGET, test_size=0.2, random_state=42
    )

    y_train_encoded, y_test_encoded, label_encoder = data_manager.encode_target(
        y_train, y_test
    )

    # --- 3. Configuración de Entrenamiento ---
    models_config = define_models_to_train()
    factory = ModelPipelineFactory()

    # Opciones comunes para RandomizedSearchCV
    SEARCH_OPTIONS = {
        'n_iter': 50,      # Número de iteraciones
        'cv': 5,           # Número de folds de validación cruzada
        'scoring': 'accuracy',
        'verbose': 1,
        'random_state': 42,
        'n_jobs': -1       # Usar todos los núcleos de CPU disponibles
    }

    # --- 3b. Configuración de MLflow ---
    mlflow.set_experiment("Pipeline_Modelado_Obesidad")
    # Activar autologging para métricas, parámetros, y el modelo de sklearn
    mlflow.sklearn.autolog(
        log_models=True,
        log_input_examples=True,
        log_model_signatures=True,
        log_datasets=False,
        disable=False,
        exclusive=False,
        log_post_training_metrics=True # Asegura que métricas post-fit se logueen
    )
    print("MLflow configurado y autologging de sklearn activado.")

    # --- 4. Bucle de Entrenamiento y Evaluación con MLflow ---
    for config in models_config:
        model_name = config["name"]
        print("="*80)
        print(f" INICIANDO ENTRENAMIENTO PARA: {model_name}")
        print("="*80)

        # Envolver cada modelo en un run de MLflow
        with mlflow.start_run(run_name=model_name) as run:

            # Entrenar el modelo usando la fábrica
            # autolog() registrará los parámetros de search_cv y las métricas de CV
            search_cv = factory.create_and_tune_model(
                X_train,
                y_train_encoded,
                config["classifier"],
                config["param_grid"],
                SEARCH_OPTIONS
            )

            best_model = search_cv.best_estimator_

            # --- 5. Resultados y Evaluación ---
            print("\n" + f"--- RESULTADOS FINALES: {model_name} ---")
            print(f"Mejores Hiperparámetros (autologged por sklearn):")
            print(search_cv.best_params_)
            print(f"\nMejor puntaje CV (Accuracy) (autologged): {search_cv.best_score_:.4f}")

            # autolog() de sklearn registra .score() automáticamente
            accuracy_test = best_model.score(X_test, y_test_encoded)
            print(f"Precisión final en Test (autologged): {accuracy_test:.4f}\n")

            # Loguear explícitamente para asegurar (aunque autolog debería tomarlo)
            mlflow.log_metric("final_test_accuracy", accuracy_test)
            mlflow.set_tag("model_type", model_name)

            # Instanciar y usar el ModelEvaluator
            # Las modificaciones en la clase loguearán las figuras y reportes
            evaluator = ModelEvaluator(
                best_model, X_test, y_test, y_test_encoded, label_encoder
            )

            print("\n--- EVALUACIÓN DETALLADA EN TEST (Logueando artefactos) ---")
            evaluator.evaluate_classification()

            print("\n--- INTERPRETACIÓN DEL MODELO (Logueando artefactos) ---")
            # autolog() también intenta loguear feature importance, pero lo hacemos
            # manualmente para asegurar que usemos nuestros nombres de features preprocesadas.
            evaluator.plot_feature_importance(X_train)

            # Lógica específica para PDP (Logueando artefactos)
            if model_name == "XGBoost":
                print('Generando gráficos de Dependencia Parcial (PDP) para XGBoost...')
                features_for_pdp = X_train.select_dtypes(include=np.number).columns.tolist()
                if features_for_pdp:
                     evaluator.plot_partial_dependence(X_train, features_for_pdp[:3])
                else:
                    print("No se encontraron features numéricas para PDP.")

            print(f"\nRun ID para {model_name}: {run.info.run_id}")

    print("="*80)
    print("Proceso de ML Completo Finalizado.")
    print("="*80)

Iniciando Proceso de ML Completo...
DataManager inicializado con el archivo: /content/drive/MyDrive/Pregrado - Posgrado - Trabajo/Maestría - Inteligencia Artificial Aplicada/11. MLOps/1. Primera etapa de proyecto/Modelado/obesity_estimation_original.csv
Iniciando carga y limpieza de datos...
Se eliminaron 24 duplicados.
No se encontraron columnas 100% nulas.
Carga y limpieza de datos completada.
Dividiendo datos en entrenamiento y prueba...
Codificando variable objetivo...
MLflow configurado y autologging de sklearn activado.
 INICIANDO ENTRENAMIENTO PARA: KNN
Preprocesador: 8 features numéricas y 8 categóricas.
Iniciando RandomizedSearchCV para KNeighborsClassifier...
Fitting 5 folds for each of 50 candidates, totalling 250 fits


2025/11/02 16:20:37 INFO mlflow.sklearn.utils: Logging the 5 best runs, 45 runs will be omitted.


Búsqueda completada para KNeighborsClassifier.

--- RESULTADOS FINALES: KNN ---
Mejores Hiperparámetros (autologged por sklearn):
{'classifier__metric': 'manhattan', 'classifier__n_neighbors': 3, 'classifier__weights': 'distance'}

Mejor puntaje CV (Accuracy) (autologged): 0.8610
Precisión final en Test (autologged): 0.8612

Evaluador listo para el modelo: KNeighborsClassifier

--- EVALUACIÓN DETALLADA EN TEST (Logueando artefactos) ---

--- Reporte de Clasificación ---
                     precision    recall  f1-score   support

Insufficient_Weight       0.82      0.94      0.88        53
      Normal_Weight       0.69      0.54      0.61        57
     Obesity_Type_I       0.86      0.93      0.89        70
    Obesity_Type_II       1.00      1.00      1.00        60
   Obesity_Type_III       0.98      1.00      0.99        65
 Overweight_Level_I       0.74      0.78      0.76        55
Overweight_Level_II       0.88      0.79      0.84        58

           accuracy                

2025/11/02 16:25:51 INFO mlflow.sklearn.utils: Logging the 5 best runs, 45 runs will be omitted.


Búsqueda completada para RandomForestClassifier.

--- RESULTADOS FINALES: Random Forest ---
Mejores Hiperparámetros (autologged por sklearn):
{'classifier__max_depth': 21, 'classifier__max_features': 'log2', 'classifier__min_samples_leaf': 2, 'classifier__min_samples_split': 6, 'classifier__n_estimators': 323}

Mejor puntaje CV (Accuracy) (autologged): 0.9377
Precisión final en Test (autologged): 0.9498

Evaluador listo para el modelo: RandomForestClassifier

--- EVALUACIÓN DETALLADA EN TEST (Logueando artefactos) ---

--- Reporte de Clasificación ---
                     precision    recall  f1-score   support

Insufficient_Weight       1.00      0.94      0.97        53
      Normal_Weight       0.79      0.95      0.86        57
     Obesity_Type_I       1.00      0.97      0.99        70
    Obesity_Type_II       1.00      1.00      1.00        60
   Obesity_Type_III       1.00      0.98      0.99        65
 Overweight_Level_I       0.91      0.89      0.90        55
Overweight_Lev

2025/11/02 16:30:25 INFO mlflow.sklearn.utils: Logging the 5 best runs, 45 runs will be omitted.


Búsqueda completada para XGBClassifier.

--- RESULTADOS FINALES: XGBoost ---
Mejores Hiperparámetros (autologged por sklearn):
{'classifier__colsample_bytree': np.float64(0.9521871356061031), 'classifier__learning_rate': np.float64(0.197306214440138), 'classifier__max_depth': 8, 'classifier__n_estimators': 233, 'classifier__subsample': np.float64(0.782613828193164)}

Mejor puntaje CV (Accuracy) (autologged): 0.9670
Precisión final en Test (autologged): 0.9737

Evaluador listo para el modelo: XGBClassifier

--- EVALUACIÓN DETALLADA EN TEST (Logueando artefactos) ---

--- Reporte de Clasificación ---
                     precision    recall  f1-score   support

Insufficient_Weight       1.00      1.00      1.00        53
      Normal_Weight       0.93      0.95      0.94        57
     Obesity_Type_I       1.00      0.97      0.99        70
    Obesity_Type_II       0.97      1.00      0.98        60
   Obesity_Type_III       1.00      0.98      0.99        65
 Overweight_Level_I       0

In [7]:
# --------------------------------------------------------------------------
# CONFIGURACIÓN
# --------------------------------------------------------------------------

# 1. Escribe el nombre EXACTO de tu experimento
EXPERIMENT_NAME = "Pipeline_Modelado_Obesidad"

# 2. Escribe la métrica EXACTA que quieres usar para comparar
#    (Debe empezar con "metrics." seguido del nombre de la métrica)
#    En tu código, guardaste la métrica de test como "final_test_accuracy"
METRIC_TO_OPTIMIZE = "metrics.final_test_accuracy"

# 3. Define si quieres el valor más alto (DESC) o más bajo (ASC)
#    Para 'accuracy' o 'AUC', usamos 'DESC' (descendente)
#    Para 'loss' o 'error', usaríamos 'ASC' (ascendente)
SORT_ORDER = "DESC"

# --------------------------------------------------------------------------
# BÚSQUEDA DEL MEJOR RUN
# --------------------------------------------------------------------------

print(f"Buscando el mejor 'Run' en el experimento: '{EXPERIMENT_NAME}'")
print(f"Optimizando para la métrica: {METRIC_TO_OPTIMIZE}\n")

try:
    # 4. Cargar el experimento
    experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
    if experiment is None:
        raise mlflow.exceptions.MlflowException(f"No se encontró el experimento '{EXPERIMENT_NAME}'")

    experiment_id = experiment.experiment_id

    # 5. Usar search_runs para obtener TODOS los runs en un DataFrame
    #    Ordenamos los resultados por la métrica deseada
    runs_df = mlflow.search_runs(
        experiment_ids=[experiment_id],
        order_by=[f"{METRIC_TO_OPTIMIZE} {SORT_ORDER}"]
    )

    if runs_df.empty:
        print("No se encontraron 'Runs' en este experimento.")
    else:
        # 6. El mejor run es la primera fila (fila 0)
        best_run = runs_df.iloc[0]

        print("--- 🏆 ¡MEJOR RUN ENCONTRADO! 🏆 ---")
        print(f"Modelo (Run Name): {best_run['tags.mlflow.runName']}")
        print(f"Run ID:            {best_run.run_id}")
        print(f"Métrica ({METRIC_TO_OPTIMIZE}): {best_run[METRIC_TO_OPTIMIZE]:.4f}")

        # 7. Mostrar los hiperparámetros de ese mejor run
        print("\n--- Hiperparámetros del Mejor Run ---")
        # Filtra todas las columnas que empiezan con "params."
        param_cols = [col for col in best_run.index if col.startswith('params.')]

        # Imprime cada parámetro
        for param in param_cols:
            param_name = param.replace('params.', '') # Nombre limpio
            param_value = best_run[param]
            print(f"{param_name}: {param_value}")

        # 8. Mostrar dónde están los artefactos (el modelo guardado y las gráficas)
        print("\n--- Ubicación de Artefactos (Modelo y Gráficas) ---")
        # El artifact_uri puede ser una ruta local o una ruta en la nube (S3, etc.)
        print(f"Ruta: {best_run.artifact_uri}")


except mlflow.exceptions.MlflowException as e:
    print(f"ERROR: No se pudo conectar o encontrar el experimento.")
    print(f"Asegúrate de que el nombre '{EXPERIMENT_NAME}' es correcto.")
    print(f"Detalle del error: {e}")

Buscando el mejor 'Run' en el experimento: 'Pipeline_Modelado_Obesidad'
Optimizando para la métrica: metrics.final_test_accuracy

--- 🏆 ¡MEJOR RUN ENCONTRADO! 🏆 ---
Modelo (Run Name): XGBoost
Run ID:            82c16bb4c9d1485cbb595a6f40825900
Métrica (metrics.final_test_accuracy): 0.9737

--- Hiperparámetros del Mejor Run ---
best_classifier__max_depth: 8
scoring: accuracy
random_state: 42
best_classifier__colsample_bytree: 0.9521871356061031
refit: True
param_distributions: {'classifier__n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7b291994b080>, 'classifier__max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7b28ecdc6630>, 'classifier__learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7b291994cec0>, 'classifier__subsample': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7b291994cd10>, 'classifier__colsample_bytree': <scipy.stats._distn_infrastructure.rv_continuous_fr